### 컨텐츠 기반 필터링(Contents-based filtering)

In [1]:
# 라이브러리 설치
!pip install scikit-surprise


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3095455 sha256=3addade496a5ec35c46be70051c2bc1344934f2a7e10aca381d0e5475efff0c6
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [23]:
import numpy as np
import pandas as pd
from surprise import Dataset

In [24]:
data = Dataset.load_builtin('ml-100k', prompt=False) # movielenz..라는 데이터
df = pd.DataFrame(data.raw_ratings, columns=['user-id', 'movie-id', 'rating', 'timestamp'])

In [25]:
df.head() # 타임스탬프(timestamp)는 영화 평점이 기록된 시간 정보

,user-id,movie-id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [26]:
df.shape, df['user-id'].nunique(), df['movie-id'].nunique()

((100000, 4), 943, 1682)

##### 1. Adjacent Matrix 생성
- 행: 사용자 id
- 열: 영화 id
- 내용: 평점(rating)

In [ ]:
# raw_ratings는 Surprise 패키지에서 제공하는 데이터셋의 속성
# 원시(raw) 형태의 평점 데이터를 담고 있는 리스트
# data.raw_ratings 
# [('196', '242', 3.0, '881250949'),
#  ('186', '302', 3.0, '891717742'),.....

In [27]:
raw_data = np.array(data.raw_ratings, dtype=int)
np.min(raw_data, axis=0) # raw_data에서 각 열(axis=0)별 최솟값 계산
# numpy는 인덱스 0부터 시작이나 data 확인시 최소값으로 1부터임을 확인

array([        1,         1,         1, 874724710])

In [9]:
raw_data

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [13]:
raw_data[:, :2] 

array([[ 196,  242],
       [ 186,  302],
       [  22,  377],
       ...,
       [ 276, 1090],
       [  13,  225],
       [  12,  203]])

- 행렬 연산: 많은 추천 시스템에서 'user-id'와 'movie-id'를 사용하여 행렬을 생성하고 연산을 수행합니다. 일부 행렬 연산 방법은 인덱스가 0부터 시작하는 것을 가정하고 설계되어 있을 수 있으므로, 데이터셋의 매핑을 0부터 시작하는 값으로 변경해야 할 수 있습니다.

- 편의성 및 호환성: 다른 라이브러리나 도구와의 호환성을 위해 'user-id'와 'movie-id'를 0부터 시작하는 값으로 변환할 수 있습니다. 이는 데이터셋을 다른 시스템 또는 도구와 통합하기 위한 편의성과 호환성을 제공할 수 있습니다.

- 이러한 경우에 'user-id'와 'movie-id'를 0부터 시작하는 값으로 변환하여 데이터를 처리하면, 특정 요구사항이나 모델의 제약 조건을 충족시키거나 데이터 처리를 보다 효율적으로 수행할 수 있습니다.

In [28]:
# user-id, movie-id가 0부터 시작하도록 만들어 줌
raw_data[:, :2] -= 1
raw_data[:5]

array([[      195,       241,         3, 881250949],
       [      185,       301,         3, 891717742],
       [       21,       376,         1, 878887116],
       [      243,        50,         2, 880606923],
       [      165,       345,         1, 886397596]])

In [19]:
np.min(raw_data, axis=0)

array([        0,         0,         1, 874724710])

##### 1) 본 영화/ 안본 영화로만 구분, 1/0

In [30]:
nrows = df['user-id'].nunique() # 943
ncols =  df['movie-id'].nunique() # 1682
adj_matrix = np.zeros([nrows, ncols], dtype=int) 
adj_matrix

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

user_id와 movie_id는 각각 사용자와 영화의 식별자로 사용됩니다. 이 값들은 데이터(raw_data)에서 반복문을 통해 가져온 사용자와 영화의 정보에 해당하는 변수입니다.

인접 행렬 adj_matrix는 사용자와 영화 간의 관계를 나타내는 행렬로, 특정 사용자와 영화의 관계를 표현하기 위해 해당 사용자의 인덱스와 영화의 인덱스를 사용합니다. 따라서 adj_matrix[user_id, movie_id]는 adj_matrix의 user_id 번째 행(row)과 movie_id 번째 열(column)에 해당하는 요소를 나타냅니다.

즉, adj_matrix[user_id, movie_id]는 adj_matrix에서 사용자와 영화의 관계를 나타내는 특정 위치의 값을 가져오거나 설정하는 것을 의미합니다.

In [34]:
# nrows = df['user-id'].nunique() # 943
# ncols =  df['movie-id'].nunique() # 1682
# adj_matrix = np.zeros([nrows, ncols], dtype=int) 
for user_id, movie_id, _, _ in raw_data:
  adj_matrix[user_id, movie_id] = 1
  # print(f"사용자 {user_id}와 영화 {movie_id}의 관계를 인접 행렬에 표시합니다.")
  # 사용자 886와 영화 471의 관계를 인접 행렬에 표시합니다.
  # 사용자 706와 영화 5의 관계를 인접 행렬에 표시합니다.
adj_matrix[:5]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [ ]:
# 여기서 row가 각 user를 의미하고 , 리스트 안에 숫자들이 영화를 봤다(1), 안봤다(0) 이걸 의미하는거에요

In [35]:
adj_matrix[0]

array([1, 1, 1, ..., 0, 0, 0])

In [37]:
# 0번 데이터를 '나'라고 가정
my_id, my_vector = 0, adj_matrix[0]
# (0, [1, 1, 1, ..., 0, 0, 0]) 이런 모습이 되는건가

- my_id는 인접 행렬 adj_matrix의 첫 번째 행을 나타내는 사용자(user)의 식별자가 됩니다.

- 따라서, my_vector는 adj_matrix의 첫 번째 행을 나타내는 벡터이며, my_id는 이 행을 나타내는 사용자의 식별자입니다.

In [36]:
# 두 배열 정의
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])

# 두 배열의 내적 계산
result = np.dot(a, b)

print(result)

32


- 위 코드에서는 a와 b라는 두 배열을 정의한 후, np.dot(a, b)를 사용하여 두 배열의 내적을 계산합니다. 이를 result 변수에 저장하고, 결과를 출력합니다.

- np.dot() 함수는 두 배열의 크기가 일치해야 하며, 배열이 1차원인 경우에는 벡터의 내적을 계산하고, 배열이 2차원인 경우에는 행렬의 곱셈을 수행합니다.

- 내적은 두 벡터의 각 요소를 곱한 후 모두 더한 값을 반환합니다. 즉, a와 b의 내적은 a[0]*b[0] + a[1]*b[1] + a[2]*b[2]와 동일합니다.

In [38]:
my_vector

array([1, 1, 1, ..., 0, 0, 0])

In [39]:
adj_matrix[10]

array([0, 0, 0, ..., 0, 0, 0])

In [40]:
# 유사도 - 이진 벡터의 내적
# 나와 10, 20번 사용자와의 유사도
np.dot(my_vector, adj_matrix[10]), np.dot(my_vector, adj_matrix[20])

(71, 42)

In [41]:
np.dot(my_vector, adj_matrix[1])

18

In [44]:
adj_matrix[]

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

In [45]:
len(adj_matrix)

943

In [46]:
# 누가 나랑 가장 닮았나?
best_score, best_match_id = 0, 0

for i in range(1, len(adj_matrix)): # 1,943 나를 제외해야해서 1부터 시작
  dot = np.dot(my_vector, adj_matrix[i])
  if dot > best_score:
    best_score, best_match_id = dot, i
  
best_score, best_match_id


(183, 275)

In [49]:
# 내가 본 영화 갯수, 가장 닮은 사람이 본 영화 갯수
my_vector.sum(), adj_matrix[best_match_id].sum()

(272, 518)

In [50]:
# 내가 보지 않은 영화중에서 가장 닮은 사람이 본 영화 --> 추천
recommend_list = []
best_vecotr = adj_matrix[best_match_id]
for i in range(len(my_vector)):
  if my_vector[i] == 0 and best_vecotr[i] == 1:
    recommend_list.append(i)
len(recommend_list), recommend_list[:10] 

(335, [272, 273, 275, 280, 281, 283, 287, 288, 289, 290])

##### 2)평점 점수를 주는경우

In [52]:
adj_matrix = np.zeros([nrows, ncols], dtype=int)
for user_id, movie_id, rating, _ in raw_data:
  adj_matrix[user_id, movie_id] = rating
adj_matrix[:5]

array([[5, 3, 4, ..., 0, 0, 0],
       [4, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [4, 3, 0, ..., 0, 0, 0]])

- Case 1) 유클리드 거리

In [53]:
# 누가 가장 '나'와 닮았나? - Euclidean distance 최소값
# 유클리드 거리가 가장 짧은것이 가장 닮은것
best_score, best_match_id = 10000000, 0
my_vector = adj_matrix[0]

for i in range(1, len(adj_matrix)):
  euc = np.sqrt(np.sum(np.square(my_vector - adj_matrix[i])))
  if euc < best_score:
    best_score, best_match_id = euc, i
  
best_score, best_match_id

(55.06359959174482, 737)

In [54]:
# 내가 보지 않은 영화중에서 가장 닮은 사람이 본 영화중 평점이 4이상인 영화 --> 추천
recommend_list = []
best_vecotr = adj_matrix[best_match_id]
for i in range(len(my_vector)):
  if my_vector[i] == 0 and best_vecotr[i] >= 4:
    recommend_list.append(i)

len(recommend_list), recommend_list[:10] 

(21, [312, 317, 356, 384, 407, 422, 433, 454, 469, 473])

- Case 2)코사인 유사도

In [55]:
def cos_similarity(v1, v2):
  v1_norm = np.sqrt(np.sum(np.square(v1)))
  v2_norm = np.sqrt(np.sum(np.square(v2)))
  return np.dot(v1, v2) / (v1_norm * v2_norm)

In [56]:
# 누가 가장 '나'와 닮았나? - Cosine similarity 최대값
best_score, best_match_id = -1, 0

for i in range(1, len(adj_matrix)):
  sim = cos_similarity(my_vector, adj_matrix[i])
  if sim > best_score:
    best_score, best_match_id = sim, i
  
best_score, best_match_id


(0.569065731527988, 915)

In [57]:
# 내가 보지 않은 영화중에서 가장 닮은 사람이 본 영화중 평점이 4이상인 영화 --> 추천
recommend_list = []
best_vecotr = adj_matrix[best_match_id]
for i in range(len(my_vector)):
  if my_vector[i] == 0 and best_vecotr[i] >= 4:
    recommend_list.append(i)

len(recommend_list), recommend_list[:10] 

(58, [275, 285, 316, 317, 381, 386, 420, 424, 426, 427])